In [1]:
!pip install -e ..

Obtaining file:///export/home/figs/vdesouza/work/Holography-1022/astrohack
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for astrohack (pyproject.toml) ... done
  Created wheel for astrohack: filename=astrohack-0.0.1-py3-none-any.whl size=1061 sha256=59f59d1e6272b2f472c5ed799d07c7c8fb2fa0681314ade2716b2f218436951c
  Stored in directory: /tmp/pip-ephem-wheel-cache-plznwbx_/wheels/93/bb/ed/46bdf801cce81c3aea69f1f4af83e386e0f89149047ab78ae0
Successfully built astrohack
  Attempting uninstall: astrohack
    Found existing installation: astrohack 0.0.1
    Uninstalling astrohack-0.0.1:
      Successfully uninstalled astrohack-0.0.1


In [2]:
import gdown
import shutil
from astrohack.panel import panel
from astrohack._utils._io import _aips_holog_to_xds
import os
from dask.distributed import Client, LocalCluster

In [3]:
# Download VLA Holography with known panel deviations
datafolder = "./paneldata/"
os.makedirs(name=datafolder, exist_ok=True)
panelzip = datafolder+"panel.zip"
if not os.path.exists(panelzip):
    url = "https://drive.google.com/u/1/uc?id=10fXyut_UHPUjIuaaEy6-m6wcycZHit2v&export=download"
    gdown.download(url, panelzip)
shutil.unpack_archive(filename=panelzip, extract_dir=datafolder)

In [4]:
# Create .zarr data set from AIPS data
holog_image = datafolder+'input.zarr'
input_data_xds = _aips_holog_to_xds(datafolder+'amp.fits', datafolder+'dev.fits')
input_data_xds.to_zarr(holog_image, mode='w')

In [5]:
%%time
# Execute Panel on a single VLA dataset
panel(holog_image, 'vla_data',
      panel_kind='corotated_robust',
      aipsdata=True, 
      telescope='vla', 
      basename='antenna25', 
      save_mask=False, 
      save_phase=False, 
      unit='mils', 
      cutoff=0.2,
      panel_margins=0.2)

CPU times: user 4.48 s, sys: 285 ms, total: 4.76 s
Wall time: 4.45 s


In [6]:
# Initialize local cluster
#asdrubal
try:
    client = Client('http://127.0.0.1:8787')

except:
    cluster = LocalCluster(
        n_workers=4,
        threads_per_worker=1,
        memory_limit='16GB'
    )

    client = Client(cluster)

/export/home/figs/vdesouza/mambaforge/envs/casadev/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38031 instead
  warnings.warn(


In [7]:
%%time
holog_image = 'hack_file'
panel_output = 'hack_file'
panel(holog_image, panel_output,
      panel_kind='corotated_robust',
      aipsdata=False, 
      telescope=None, 
      save_mask=False, 
      save_phase=False, 
      unit='mm', cutoff=0.2)

[2023-03-20 14:25:17.260761 System Info]: Processing ddi_0 for ant_3
CPU times: user 738 ms, sys: 73.3 ms, total: 812 ms
Wall time: 15.8 s
